### Data Preprocessing

In [13]:
# Data Manipulation Libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Plotting Libraries
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

store_df = pd.read_csv("../data/store.csv")
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')
submission_df = pd.read_csv('../data/sample_submission.csv')

In [33]:
# Merge store df and test df
test_merged_df = test_df.merge(store_df, how='left', on='Store')

# Merge store df and train df for better prediction
train_merged_df = train_df.merge(store_df, how='left', on='Store')
train_merged_df = train_merged_df.drop(['PromoInterval'], axis=1)

# Replacing string '0' as integer '0' 
train_merged_df['StateHoliday'].replace({'0': 0}, inplace=True)
test_merged_df['StateHoliday'].replace({'0': 0}, inplace=True)

In [34]:
""" 
Preparing Test Datasets:

1) Access to test.csv data and merging it with store.csv
2) Transforming Datetime string column to pandas datetime for better access to Date column
3) Create 3 new columns named year, month, day similar to train dataset. 

"""

test_merged_df['Date'] = pd.to_datetime(test_merged_df.Date)

test_merged_df['Year'] = test_merged_df.Date.dt.year
test_merged_df['Month'] = test_merged_df.Date.dt.month
test_merged_df['Day'] = test_merged_df.Date.dt.day

print(f"Estimate Sales from {test_merged_df.Date.dt.date.min()} to {test_merged_df.Date.dt.date.max()}")
test_merged_df

Estimate Sales from 2015-08-01 to 2015-09-17


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day
0,1,1,4,2015-09-17,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015,9,17
1,2,3,4,2015-09-17,1.0,1,0,0,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",2015,9,17
2,3,7,4,2015-09-17,1.0,1,0,0,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN,2015,9,17
3,4,8,4,2015-09-17,1.0,1,0,0,a,a,7520.0,10.0,2014.0,0,NaN,NaN,NaN,2015,9,17
4,5,9,4,2015-09-17,1.0,1,0,0,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN,2015,9,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41083,41084,1111,6,2015-08-01,1.0,0,0,0,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct",2015,8,1
41084,41085,1112,6,2015-08-01,1.0,0,0,0,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN,2015,8,1
41085,41086,1113,6,2015-08-01,1.0,0,0,0,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN,2015,8,1
41086,41087,1114,6,2015-08-01,1.0,0,0,0,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN,2015,8,1


In [42]:

reduced_train_df = train_merged_df[ train_merged_df.Open == 1 ].copy()

# Convert 'Date' column to datetime format
reduced_train_df['Date'] = pd.to_datetime(reduced_train_df['Date'])

# Extracting year and month from the 'Date' column
reduced_train_df['Year'] = reduced_train_df['Date'].dt.year
reduced_train_df['Month'] = reduced_train_df['Date'].dt.month
reduced_train_df['Day'] = reduced_train_df['Date'].dt.day

print(f"Estimate Sales data from {reduced_train_df.Date.dt.date.min()} to {reduced_train_df.Date.dt.date.max()}")

Estimate Sales data from 2013-01-01 to 2015-07-31


In [43]:
train_df = reduced_train_df[reduced_train_df.Date.dt.year <= 2014]
val_df = reduced_train_df[reduced_train_df.Date.dt.year == 2015]

print(f"Training Shape: {train_df.shape}")
print(f"Validation Shape: {val_df.shape}")
print(f"Test Shape: {test_merged_df.shape}")

Training Shape: (648360, 20)
Validation Shape: (196032, 20)
Test Shape: (41088, 20)


### Data Preprocessing

In [44]:
# choose columns to train and target

# Set columns for training
input_cols = ['Store', 'DayOfWeek', 'Promo', 'StateHoliday', 'StoreType', 'Assortment', 'Promo2', 'Day', 'Month', 'Year']
# Set Target Column
target_col = 'Sales'

train_inputs = train_df[input_cols].copy()
train_targets = train_df[target_col].copy()

val_inputs = val_df[input_cols].copy()
val_targets = val_df[target_col].copy()

test_inputs = test_merged_df[input_cols].copy()

num_cols = ['Store', 'DayOfWeek', 'Day', 'Month', 'Year']
cat_cols = ['DayOfWeek', 'Promo', 'StoreType', 'Assortment', 'Promo2']

In [45]:
# Encoding

train_inputs = pd.get_dummies(train_inputs)
val_inputs = pd.get_dummies(val_inputs, drop_first=False)
test_inputs = pd.get_dummies(test_inputs)

# Drop the columns that are not in test dataset
train_inputs = train_inputs.drop(['StateHoliday_b', 'StateHoliday_c'], axis=1)
val_inputs = val_inputs.drop(['StateHoliday_b'], axis=1)

In [46]:
# Scaling

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(train_inputs[num_cols])

# Based on the learning from training datasets, val_df and test_df columns sclaed!
train_inputs[num_cols] = scaler.transform(train_inputs[num_cols])
val_inputs[num_cols] = scaler.transform(val_inputs[num_cols])
test_inputs[num_cols] = scaler.transform(test_inputs[num_cols])

### choose a model

In [47]:
"""
As the prediction will be evaluated on RMSPE metric, Create a user-defined
rmspe function. 

"""

def rmspe(y_true, y_pred):
    """
    Compute the Root Mean Square Percentage Error (RMSPE) between the true and predicted values.
    
    Parameters:
    - y_true: array-like, true target values
    - y_pred: array-like, predicted target values
    
    Returns:
    - float, RMSPE value
    """
    # Ensure both arrays have the same length
    assert len(y_true) == len(y_pred)
    
    # Compute the percentage error for each observation
    percentage_error = (y_true - y_pred) / y_true
    
    # Exclude observations where true value is zero
    percentage_error[y_true == 0] = 0
    
    # Square the percentage errors
    squared_percentage_error = percentage_error ** 2
    
    # Compute the mean of the squared percentage errors
    mean_squared_percentage_error = np.mean(squared_percentage_error)
    
    # Compute the square root of the mean squared percentage error
    rmspe = np.sqrt(mean_squared_percentage_error)
    
    return rmspe # Convert to percentage

In [48]:
# A helper function to automate task of fitting model and evaluating

def try_model(model):
    model.fit(train_inputs, train_targets)

    train_preds = model.predict(train_inputs)
    val_preds = model.predict(val_inputs)

    # Get RMSE
    train_rmse = np.round(mean_squared_error(train_targets, train_preds, squared=False), 5)
    val_rmse = np.round(mean_squared_error(val_targets, val_preds, squared=False), 5)

    # Get RMSPE
    train_rmspe = np.round(rmspe(train_targets, train_preds), 5)
    val_rmspe = np.round(rmspe(val_targets, val_preds), 5)


    print(f"Train RMSE: {train_rmse}")
    print(f"Val RMSE: {val_rmse}")
    print()
    print(f"Train RMSPE: {train_rmspe}")
    print(f"Val RMSPE: {val_rmspe}")

    return model

In [49]:
# Creating a dumb model that can predict average sales only
        
def get_mean(input):
    return np.full(len(input), reduced_train_df.Sales.mean())

dum_train_preds = get_mean(train_df)
dum_val_preds = get_mean(val_df)

from sklearn.metrics import mean_squared_error

# Get root mean squared error
dum_train_eval = mean_squared_error(dum_train_preds, train_targets, squared=False)
dum_val_eval = mean_squared_error(dum_val_preds, val_targets, squared=False)

train_rmspe = rmspe(dum_train_preds, train_targets)
val_rmspe = rmspe(dum_val_preds, val_targets)

print(f"train_rmse: {dum_train_eval}")
print(f"val_rmse: {dum_val_eval}")
print()
print(f"train_rmspe: {train_rmspe}")
print(f"val_rmspe: {val_rmspe}")

train_rmse: 3124.212911034433
val_rmse: 3037.1262166880488

train_rmspe: 0.44917065517162236
val_rmspe: 0.4366501296280081


### Linear Regression as a Baseline Model

In [53]:
# Machine Learning Libraries
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [54]:
logit_model = LinearRegression()
try_model(logit_model)

Train RMSE: 2787.2088
Val RMSE: 2694.56185

Train RMSPE: 0.5379
Val RMSPE: 0.47785


LinearRegression()

### Decision Tree Regressor

In [55]:
decision_tree_model = DecisionTreeRegressor(random_state=42)
try_model(decision_tree_model)

Train RMSE: 0.0
Val RMSE: 1380.17354

Train RMSPE: 0.0
Val RMSPE: 0.21355


DecisionTreeRegressor(random_state=42)

### Random Forest Model

In [56]:
random_forest_model = RandomForestRegressor(random_state=42, n_jobs=-1)
try_model(random_forest_model)

Train RMSE: 400.79893
Val RMSE: 1134.79363

Train RMSPE: 0.09413
Val RMSPE: 0.1785


RandomForestRegressor(n_jobs=-1, random_state=42)

### Post-Prediction Analysis

In [58]:
# Feature importance
feature_importance_df = pd.DataFrame({ 
    'feature': train_inputs.columns, 
    'importance': random_forest_model.feature_importances_
}).sort_values('importance', ascending=False)
feature_importance_df

,feature,importance
0,Store,0.565438
2,Promo,0.134409
1,DayOfWeek,0.067784
4,Day,0.059565
5,Month,0.056752
10,StoreType_b,0.023134
3,Promo2,0.015872
9,StoreType_a,0.014520
15,Assortment_c,0.013404
6,Year,0.011328


### Model Serialization

In [59]:
import joblib
from datetime import datetime

# Serialize the model with a timestamp
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
joblib.dump(random_forest_model, f"model_{timestamp}.pkl")

['model_2024-09-23-10-21-57.pkl']

In [61]:
import pickle

# Load the model
with open('model_2024-09-23-10-21-57.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

### submission

In [64]:
test_preds = random_forest_model.predict(test_inputs)
submission_df = pd.read_csv('../data/sample_submission.csv')
submission_df['Sales'] = test_preds
submission_df.to_csv('../data/submission.csv', index=False)
test_preds

### Time Series Preprocessing

In [22]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Preprocess time series data for LSTM
sales_data = train_df[['Date', 'Sales']].set_index('Date').sort_index()
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_sales = scaler.fit_transform(sales_data)

# Sliding window to create supervised learning data
def create_supervised(data, look_back=1):
    X, y = [], []
    for i in range(len(data)-look_back-1):
        X.append(data[i:(i+look_back), 0])
        y.append(data[i + look_back, 0])
    return np.array(X), np.array(y)

look_back = 7
X, y = create_supervised(scaled_sales, look_back)

In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(look_back, 1)))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

# Fit the model
model.fit(X, y, epochs=1, batch_size=32)

Epoch 1/10
31788/31788 [==============================] - 298s 9ms/step - loss: 0.0171
Epoch 2/10
31788/31788 [==============================] - 377s 12ms/step - loss: 0.0167
Epoch 3/10
31788/31788 [==============================] - 384s 12ms/step - loss: 0.0166
Epoch 4/10
31788/31788 [==============================] - 382s 12ms/step - loss: 0.0165
Epoch 5/10
31788/31788 [==============================] - 390s 12ms/step - loss: 0.0164
Epoch 6/10
31788/31788 [==============================] - 399s 13ms/step - loss: 0.0163
Epoch 7/10
31788/31788 [==============================] - 397s 13ms/step - loss: 0.0162
Epoch 8/10
31788/31788 [==============================] - 404s 13ms/step - loss: 0.0162
Epoch 9/10
31788/31788 [==============================] - 408s 13ms/step - loss: 0.0161
Epoch 10/10
31788/31788 [==============================] - 404s 13ms/step - loss: 0.0161


In [28]:
from fastapi import FastAPI
import joblib
import numpy as np

app = FastAPI()

# Load model on startup
@app.on_event("startup")
async def load_model():
    global model
    model = joblib.load('model_2024-09-19.pkl')

# Define API for prediction
@app.post("/predict/")
async def predict(features: list):
    input_features = np.array([features])
    prediction = model.predict(input_features)
    return {"prediction": prediction.tolist()}